In [1]:
# #comment this if you are not using AIT proxy...
# import os
# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'

In [2]:
import os
# Set GPU device
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
class argument:
    def __init__(self):
        self.dataset_name = 'wikitext'
        self.dataset_config_name = 'wikitext-2-raw-v1'
        self.output_dir = './logs/' 
        self.seed = 1234
        self.learning_rate = 5e-5
        self.block_size = 1024 
        self.do_ref_model = False
        
        self.config_name = None
        self.model_name_or_path = 'gpt2'
        self.tokenizer_name = 'gpt2'
        self.use_slow_tokenizer = False
        
        self.per_device_train_batch_size = 8
        self.per_device_eval_batch_size = 8
        self.gradient_accumulation_steps = 8
        
        self.do_ref_model = False
        self.lr_scheduler_type = 'linear'

        self.num_train_epochs = 5
        self.max_train_steps = None

        self.preprocessing_num_workers = 1
        self.overwrite_cache = False
        self.weight_decay = 0.0
        self.num_warmup_steps = 0
        
        self.add_canary = True
        self.canary_rep = 50
        self.canary_len = 5
        
        self.add_adapter = False
        self.adapter_reduction = 16
        self.train_head_only = False
        self.train_layer_n_only = None 
        self.redact_token = 'multi'
        
        
args = argument()

In [4]:
from enum import unique
import logging
import math
import os
import random
from itertools import chain
from pathlib import Path
import copy 
from sys import path
import sys
# from utils import Logger


import datasets
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

import transformers
from accelerate import Accelerator, DistributedType
from huggingface_hub import Repository
from transformers import (
#    CONFIG_MAPPING,
#    MODEL_MAPPING,
    AdamW,
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    SchedulerType,
    default_data_collator,
    get_scheduler,
    set_seed,
)

#from torch import AdamW
from transformers.utils.versions import require_version
import datasets
from datasets import load_dataset
from random import shuffle
import numpy as np
import random
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import csv
from scipy.stats import skewnorm
from scipy.stats import kstest

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
random.seed(args.seed)

folder_name = f"canary_{str(args.canary_rep)}_{str(args.canary_len)}_adapter_{args.add_adapter}_head_{args.train_head_only}_layer_{args.train_layer_n_only}_ref_{args.do_ref_model}_maxlen_{args.block_size}_red_{args.adapter_reduction}_model_{args.model_name_or_path}_lr_{args.learning_rate}_epoch_{args.num_train_epochs}_trba_{args.per_device_train_batch_size}_acc_{args.gradient_accumulation_steps}_evba{args.per_device_eval_batch_size}_data_{args.dataset_name}"

directory = "{}/{}".format(args.output_dir,folder_name)
print(directory)
if not os.path.exists(directory):
    os.mkdir(directory)

# log_file = os.path.join(directory, "stdout")

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# if accelerator.is_local_main_process:
    # print("Logging to {}".format(log_file))
    # pass
    
# sys.stdout = Logger(log_file)

./logs//canary_50_5_adapter_False_head_False_layer_None_ref_False_maxlen_1024_red_16_model_gpt2_lr_5e-05_epoch_5_trba_8_acc_8_evba8_data_wikitext


In [6]:
if args.dataset_name is not None:
    # Downloading and loading a dataset from the hub.
    if 'enron' in args.dataset_name:
        raw_datasets = load_dataset('csv', data_files={'train': 'data/cleaned_short_train_scrubbed.csv' ,'validation': 'data/cleaned_short_test_scrubbed.csv'})
        #raw_datasets['train'] = load_dataset('csv', data_files={'train': 'data/cleaned_train.csv' ,'validation': 'data/cleaned_test.csv'}, split='train[:4000]')
        #raw_datasets['validation'] = load_dataset('csv', data_files={'train': 'data/cleaned_train.csv' ,'validation': 'data/cleaned_test.csv'}, split='train[4000:5000]')

    else:
        raw_datasets = load_dataset(args.dataset_name, args.dataset_config_name)
        if "validation" not in raw_datasets.keys():
            raw_datasets["validation"] = load_dataset(
                args.dataset_name,
                args.dataset_config_name,
                split=f"train[:{args.validation_split_percentage}%]",
            )
            raw_datasets["train"] = load_dataset(
                args.dataset_name,
                args.dataset_config_name,
                split=f"train[{args.validation_split_percentage}%:]",
            )
else:
        data_files = {}
        dataset_args = {}
        if args.train_file is not None:
            data_files["train"] = args.train_file
        if args.validation_file is not None:
            data_files["validation"] = args.validation_file
        extension = args.train_file.split(".")[-1]
        if extension == "txt":
            extension = "text"
            dataset_args["keep_linebreaks"] = not args.no_keep_linebreaks
        raw_datasets = load_dataset(extension, data_files=data_files, **dataset_args)
        # If no validation data is there, validation_split_percentage will be used to divide the dataset.
        if "validation" not in raw_datasets.keys():
            raw_datasets["validation"] = load_dataset(
                extension,
                data_files=data_files,
                split=f"train[:{args.validation_split_percentage}%]",
                **dataset_args,
            )
            raw_datasets["train"] = load_dataset(
                extension,
                data_files=data_files,
                split=f"train[{args.validation_split_percentage}%:]",
                **dataset_args,
            )

Found cached dataset wikitext (/home/todsavadt/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
100%|██████████| 3/3 [00:00<00:00, 2135.23it/s]


In [7]:
raw_datasets

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [8]:
raw_datasets['train']['text'][3]

' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n'

In [9]:
import re
# Define the regular expressions
def clean_text(example):
    text = example["text"]
    text = re.sub(r'@\.@', '.', text)
    text = re.sub(r'@,@', ',', text)
    text = re.sub(r'@-@', '-', text)
    example["text"] = text
    return example

# Apply the regular expressions to the train and validation datasets
for split in ["train", "validation"]:
    raw_datasets[split] = raw_datasets[split].map(clean_text)
raw_datasets

Loading cached processed dataset at /home/todsavadt/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-e9d61d49ce43d74a.arrow
Loading cached processed dataset at /home/todsavadt/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-eb8c1efa36848e7e.arrow


DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [10]:
raw_datasets['train']['text'][3]

' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role - playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real - time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n'

In [11]:
if args.config_name:
    config = AutoConfig.from_pretrained(args.config_name)
elif args.model_name_or_path:
    config = AutoConfig.from_pretrained(args.model_name_or_path)
#    else:
#        config = CONFIG_MAPPING[args.model_type]()
#        logger.warning("You are instantiating a new config instance from scratch.")

if args.tokenizer_name:
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, use_fast=not args.use_slow_tokenizer)
elif args.model_name_or_path:
    tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=not args.use_slow_tokenizer)
else:
    raise ValueError(
        "You are instantiating a new tokenizer from scratch. This is not supported by this script."
        "You can do it from another script, save it, and load it from here, using --tokenizer_name."
    )

### Modify Model

In [12]:
# if args.model_name_or_path:
#     model = AutoModelForCausalLM.from_pretrained(
#         args.model_name_or_path,
#         from_tf=bool(".ckpt" in args.model_name_or_path),
#         config=config,
#     )
# else:
#     # logger.info("Training new model from scratch")
#     model = AutoModelForCausalLM.from_config(config)

# model.resize_token_embeddings(len(tokenizer))

# # model_ref = copy.deepcopy(model)

In [13]:
import torch
import torch.nn as nn

class CustomGPT2HeadModel(nn.Module):
    def __init__(self, config):
        super(CustomGPT2HeadModel, self).__init__()
        self.transformer = AutoModelForCausalLM.from_pretrained(
                                args.model_name_or_path,
                                from_tf=bool(".ckpt" in args.model_name_or_path),
                                config=config,
                            )
        self.pv_embed    = nn.Embedding(2, config.n_embd)
        self.alpha       = 0.8
   
    def forward(self, 
                input_ids, 
                private_ids=None, 
                attention_mask=None, 
                labels = None):
        
        # Get token embeddings from GPT-2
        inputs_embeds = self.transformer.transformer.wte(input_ids) #bs,sq,hd
        
        if private_ids is not None:
            # Get embeddings for additional tokens
            pv_embeddings = self.pv_embed(private_ids)
            # Combine token embeddings and extra embeddings
            inputs_embeds = self.alpha * inputs_embeds + (1 - self.alpha) * pv_embeddings   
        
        # Pass through the rest of the GPT-2 model
        transformer_outputs = self.transformer(
            # input_ids = input_ids,
            inputs_embeds = inputs_embeds, 
            attention_mask = attention_mask,
            labels = labels)
        
        return transformer_outputs

args.customgpt = True
if args.customgpt:
    # Example usage
    # config = GPT2Config.from_pretrained('distilgpt2')
    model = CustomGPT2HeadModel(config)
    # model.resize_token_embeddings(len(tokenizer))

In [14]:
#Initializae Weight
model.pv_embed.weight

Parameter containing:
tensor([[ 0.9914,  0.4752, -2.3984,  ...,  0.8599, -0.4503, -0.9561],
        [ 1.3936,  0.3157,  0.6289,  ..., -0.1873,  1.3695,  1.5500]],
       requires_grad=True)

In [15]:
import torch
import torch.nn as nn

class BinaryEmbedding(nn.Module):
    def __init__(self):
        super(BinaryEmbedding, self).__init__()
        self.embedding_dim = 768
        
        # Create a single embedding for the binary variable
        self.embedding = nn.Embedding(2, self.embedding_dim)
        
    def forward(self, 
                private_ids=None, 
                input_ids=None, 
                attention_mask=None, 
                labels=None):
        # Convert input indices to binary vectors (0 or 1)
        binary_input = torch.where(private_ids == 0, torch.tensor(0), torch.tensor(1))
        binary_label = torch.where(labels == 0, torch.tensor(0), torch.tensor(1))
        
        # Get embeddings for the binary vectors
        binary_embeddings = self.embedding(binary_input)
        binary_embeddings_label = self.embedding(binary_label)
        return binary_embeddings, binary_embeddings_label

In [16]:
#Load Weight Outside
binary_embedding_model = BinaryEmbedding()
binary_embedding_model.load_state_dict(torch.load('binary_embedding_model.pth'))

#Initializae Weight
model.pv_embed.weight = binary_embedding_model.embedding.weight
model.pv_embed.weight

Parameter containing:
tensor([[ 0.5892, -1.0553,  0.9204,  ..., -0.4016, -0.2926, -0.5933],
        [ 0.3032,  0.0427,  0.3495,  ..., -1.1115,  0.1880,  0.1024]],
       requires_grad=True)

### Entity

In [17]:
import spacy
NLP = spacy.load("en_core_web_sm")

MASK_TOKEN = "<MASK>"

# can be found here, https://github.com/explosion/spaCy/blob/master/spacy/glossary.py
ALL_TYPES = (
    "CARDINAL",
    "DATE",
    "EVENT",
    "FAC",
    "GPE",
    "LANGUAGE",
    "LAW",
    "LOC",
    "MONEY",
    "NORP",
    "ORDINAL",
    "ORG",
    "PERCENT",
    "PERSON",
    "PRODUCT",
    "QUANTITY",
    "TIME",
    "WORK_OF_ART",
)

SPECIAL_TOKENS_MAP = {
    # dep parser
    "SUBJ": "<SUBJ>",
    "OBJ": "<OBJ>",
    "ROOT": "<ROOT>",
    # pos tagging
    "PROPN": "<PROPN>",
    "PRON": "<PRON>",
    # SRL predicate
    "VERB": "<VERB>",
    "MASK": "<MASK>",
}

for ent_type_ in ALL_TYPES:
    SPECIAL_TOKENS_MAP.update({ent_type_: f"<{ent_type_.upper()}>"})


# len(ALL_TYPES)

def get_spacy_tokens_and_doc(line):
    doc = NLP(line)
    spacy_tokens = [x.text for x in doc]
    return spacy_tokens, doc
    
def get_special_tokens(special_token, use_single_mask_token=True):
    use_single_mask_token = True if args.redact_token == 'single' else False
    special_token = special_token.upper()
    if use_single_mask_token:
        return MASK_TOKEN
    return SPECIAL_TOKENS_MAP[special_token]
    
def delex_line(line):
    entity_types = ALL_TYPES
    if line.endswith("\n"):
        endswith_new_line = True
        line = line[:-1]
        assert not line.endswith("\n"), "line still ends with \n"
    else:
        endswith_new_line = False
    _, doc = get_spacy_tokens_and_doc(line.strip())
    words = [tok.text for tok in doc]
    spaces = [True if tok.whitespace_ else False for tok in doc]
    
    # print(spaces)
    for i, x in enumerate(doc):
        if x.ent_type_ in entity_types:
            # named entity
            words[i] = get_special_tokens(x.ent_type_)
            need_to_add = True
    total = len(doc)

    # rejoin them
    doc2 = spacy.tokens.doc.Doc(NLP.vocab, words=words, spaces=spaces)
    return_text = doc2.text
    if endswith_new_line:
        return_text = return_text + "\n"
    return return_text

def delex_line_digit(line):
    entity_types = ALL_TYPES
    if line.endswith("\n"):
        endswith_new_line = True
        line = line[:-1]
        assert not line.endswith("\n"), "line still ends with \n"
    else:
        endswith_new_line = False
    _, doc = get_spacy_tokens_and_doc(line.strip())
    words = [tok.text for tok in doc]
    # spaces = [True if tok.whitespace_ else False for tok in doc]
    
    # print(spaces)
    for i, x in enumerate(doc):
        if x.ent_type_ in entity_types:
            # named entity
            words[i] = 1 #get_special_tokens(x.ent_type_, use_single_mask_token=True)
            need_to_add = True
        else:
            words[i] = 0
    total = len(doc)

    if endswith_new_line:
        words.append(0)
    return words

### Add canary

In [18]:
def gen_canary(canary_len,tokenizer):
    raw_sample = random.choices([str(i) for i in range(10)], k=canary_len)
    raw_sample = " ".join(raw_sample)
    
    tokenized = tokenizer.tokenize(raw_sample)
    ids = tokenizer.convert_tokens_to_ids(tokenized)
    assert len(ids) == canary_len
    
    raw_sample = "the secret number is " + raw_sample
    toked =  tokenizer(raw_sample)
    toked['private_ids'] = delex_line_digit(raw_sample)
    toked['labels'] = toked['input_ids'].copy()
    return raw_sample, toked

In [19]:
if args.add_canary:    
    if 'ptb' in args.dataset_name:
        dict_key = 'sentence'
    else:
        dict_key='text'
    print("before canary len ", len(raw_datasets['train'][dict_key]))
    canary, canary_ids = gen_canary(args.canary_len, tokenizer)
    for j in range(args.canary_rep):
        raw_datasets['train']=raw_datasets['train'].add_item({dict_key:canary})

    raw_datasets['train'] = raw_datasets['train'].shuffle(seed=args.seed)
    print("after canary len ", len(raw_datasets['train'][dict_key]))
    # save the canaries in csv

    file = open(f'./{directory}/canaries.txt', 'w+')
    file.write(canary)
    file.write('\n')
    file.close()

    file = open(f'./{directory}/fitting_canaries.txt', 'w+')
    
    fitting_canaries_ids = []
    for i in range(5000):
        fit , fit_ids = gen_canary(args.canary_len,tokenizer)
        if fit != canary:
            fitting_canaries_ids.append(fit_ids)
            file.write(fit)
            file.write('\n')
    print(len(fitting_canaries_ids))

Loading cached shuffled indices for dataset at /home/todsavadt/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-cc9c7d9f3ffada8e.arrow


before canary len  36718
after canary len  36768
5000


In [20]:
# canary, delex_line(canary), delex_line_digit(canary)
canary, canary_ids

('the secret number is 9 4 0 9 9',
 {'input_ids': [1169, 3200, 1271, 318, 860, 604, 657, 860, 860], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1], 'private_ids': [0, 0, 0, 0, 1, 1, 1, 1, 1], 'labels': [1169, 3200, 1271, 318, 860, 604, 657, 860, 860]})

In [21]:
# train_private_token = [delex_line_digit(line) for line in tqdm(raw_datasets['train']['text'])]
# validation_private_token = [delex_line_digit(line) for line in tqdm(raw_datasets['validation']['text'])]

In [22]:
import pickle
file_path = "masker/train_private_token_wikitext.pkl"
# with open(file_path, "wb") as file:
#     pickle.dump(train_private_token, file)
with open(file_path, "rb") as file:
    train_private_token = pickle.load(file)

file_path = "masker/validation_private_token_wikitext.pkl"
# with open(file_path, "wb") as file:
#     pickle.dump(validation_private_token, file)
with open(file_path, "rb") as file:
    validation_private_token = pickle.load(file)

In [23]:
from datasets import DatasetDict, Dataset
# Create new dictionaries with both 'text' and 'labels' features
new_train_dict = {
    "text": raw_datasets["train"]["text"],
    "private_ids": train_private_token,
}
new_validation_dict = {
    "text": raw_datasets["validation"]["text"],
    "private_ids": validation_private_token,
}
# new_test_dict = {
#     "text": raw_datasets["test"]["text"],
#     # "private_ids": test_private_token,
# }

# Create a new DatasetDict with the additional 'labels' feature
new_dataset = DatasetDict({
    "train": Dataset.from_dict(new_train_dict),
    "validation": Dataset.from_dict(new_validation_dict),
    # "test": Dataset.from_dict(new_test_dict),
})

new_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'private_ids'],
        num_rows: 36768
    })
    validation: Dataset({
        features: ['text', 'private_ids'],
        num_rows: 3760
    })
})

In [24]:
new_dataset['train']['text'][3], new_dataset['train']['private_ids'][3]

(' 2nd & 5th Detail Issue Depots , RASC \n', [1, 1, 0, 1, 0, 1, 0, 1, 0])

In [25]:
# Preprocessing the datasets.
# First we tokenize all the texts.
column_names = raw_datasets["train"].column_names
text_column_name = "text" if "text" in column_names else column_names[0]

def tokenize_function(examples):
    return tokenizer(examples[text_column_name])

with accelerator.main_process_first():
    tokenized_datasets = new_dataset.map(
        tokenize_function,
        batched=True,
        num_proc=args.preprocessing_num_workers,
        remove_columns=column_names,
        load_from_cache_file=not args.overwrite_cache,
        desc="Running tokenizer on dataset",
    )

if args.block_size is None:
    block_size = tokenizer.model_max_length
    if block_size > 1024:
        print(
            f"The tokenizer picked seems to have a very large `model_max_length` ({tokenizer.model_max_length}). "
            "Picking 1024 instead. You can change that default value by passing --block_size xxx."
        )
    block_size = 1024
else:
    if args.block_size > tokenizer.model_max_length:
        print(
            f"The block_size passed ({args.block_size}) is larger than the maximum length for the model"
            f"({tokenizer.model_max_length}). Using block_size={tokenizer.model_max_length}."
        )
    block_size = min(args.block_size, tokenizer.model_max_length)

In [26]:
# Main data processing function that will concatenate all texts from our dataset and generate chunks of block_size.
def group_texts(examples):
    # Concatenate all texts.
    
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result
    
# Note that with `batched=True`, this map processes 1,000 texts together, so group_texts throws away a remainder
# for each of those groups of 1,000 texts. You can adjust that batch_size here but a higher value might be slower
# to preprocess.
#
# To speed up this part, we use multiprocessing. See the documentation of the map method for more information:
# https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map

with accelerator.main_process_first():
    lm_datasets = tokenized_datasets.map(
        group_texts,
        batched=True,
        num_proc=args.preprocessing_num_workers,
        load_from_cache_file=not args.overwrite_cache,
        desc=f"Grouping texts in chunks of {block_size}",
    )

train_dataset = lm_datasets["train"]
eval_dataset = lm_datasets["validation"]

In [27]:
train_dataset, eval_dataset

(Dataset({
     features: ['private_ids', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 2010
 }),
 Dataset({
     features: ['private_ids', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 210
 }))

In [28]:
small_train_dataset = train_dataset.shuffle(seed=55).select(range(20))
small_eval_dataset = eval_dataset.shuffle(seed=55).select(range(20))
small_train_dataset, small_eval_dataset

(Dataset({
     features: ['private_ids', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 20
 }),
 Dataset({
     features: ['private_ids', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 20
 }))

In [29]:
# Log a few random samples from the training set:
#for index in random.sample(range(len(train_dataset)), 3):
#    logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")
args.small_sample = False
if args.small_sample:
    # DataLoaders creation:
    train_dataloader = DataLoader(
        small_train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=args.per_device_train_batch_size
    )
    eval_dataloader = DataLoader(
        small_eval_dataset, collate_fn=default_data_collator, batch_size=args.per_device_eval_batch_size
    )
else:
    # DataLoaders creation:
    train_dataloader = DataLoader(
        train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=args.per_device_train_batch_size
    )
    eval_dataloader = DataLoader(
        eval_dataset, collate_fn=default_data_collator, batch_size=args.per_device_eval_batch_size
    )

In [30]:
#checking chucking
for batch in train_dataloader:
    print(batch.keys())
    print(batch['input_ids'].shape, batch['labels'].shape)
    break
for batch in eval_dataloader:
    print(batch['input_ids'].shape, batch['labels'].shape)
    break

dict_keys(['private_ids', 'input_ids', 'attention_mask', 'labels'])
torch.Size([8, 1024]) torch.Size([8, 1024])
torch.Size([8, 1024]) torch.Size([8, 1024])


In [31]:
# output = model(**batch)
# output.loss

In [32]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

#print(model.lm_head)    
if accelerator.is_local_main_process:
    print("model_params (million)", count_parameters(model)/1000000)

model_params (million) 124.441344


In [33]:
 # Split weights in two groups, one with weight decay and the other not.
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate)

/home/todsavadt/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [34]:
# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

# model_ref = accelerator.prepare(
#     model_ref
# )

In [35]:
# On TPU, the tie weights in our model have been disconnected, so we need to restore the ties.
if accelerator.distributed_type == DistributedType.TPU:
    model.tie_weights()

# Note -> the training dataloader needs to be prepared before we grab his length below (cause its length will be
# shorter in multiprocess)

# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)

if args.max_train_steps is None:
    args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
else:
    args.num_train_epochs = math.ceil(args.max_train_steps / num_update_steps_per_epoch)

lr_scheduler = get_scheduler(
    name=args.lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=args.num_warmup_steps,
    num_training_steps=args.max_train_steps,
)

# Train!
total_batch_size = args.per_device_train_batch_size * accelerator.num_processes * args.gradient_accumulation_steps

In [36]:
if accelerator.is_local_main_process:
    print("model_params (million)", count_parameters(model)/1000000)

model_params (million) 124.441344


In [37]:
# def calculate_exposure(canary_rank):
    # return math.log(TOTAL_CANDIDATES, 2) - math.log(canary_rank, 2)

In [38]:
def get_exposure(fitting, main):

    fitting_params = skewnorm.fit(fitting)
    ks = kstest(fitting, 'skewnorm', fitting_params)

    cdf = skewnorm.cdf(main, fitting_params[0], fitting_params[1], fitting_params[2])

    if cdf == 0.0:
        exposure = 0.0
    else:
        exposure = -1.0*np.log2(cdf)

    return exposure

def get_fit_canary_loss(model,fitting_id, main_id):
    loss_list = []
    for k, v in main_id.items():
            main_id[k] = torch.tensor(v).cuda()

    input_ids = main_id['input_ids']
    labels = main_id['labels']

    loss_main = np.exp(model(input_ids=input_ids, labels=labels)['loss'].item())
    # loss_main = np.exp(model(**main_id)['loss'].item())

    for sample in fitting_id:
        for k, v in sample.items():
            sample[k] = torch.tensor(v).cuda()
        
        input_ids = sample['input_ids']
        labels = sample['labels']
        output = model(input_ids=input_ids, labels=labels)
        # output = model(**sample)
        loss_list.append(np.exp(output.loss.item()))

    return loss_main, loss_list

def gen_canary(canary_len, tokenizer):
    raw_sample = random.choices([str(i) for i in range(10)], k=canary_len)
    raw_sample = " ".join(raw_sample)
    
    tokenized = tokenizer.tokenize(raw_sample)
    ids = tokenizer.convert_tokens_to_ids(tokenized)
    assert len(ids) == canary_len
    
    raw_sample = "the secret number is " + raw_sample
    toked =  tokenizer(raw_sample)
    toked['labels'] = toked['input_ids'].copy()
    return raw_sample, toked

In [39]:
print("***** Running training *****")
print(f"  Num examples = {len(train_dataset)}")
print(f"  Num Epochs = {args.num_train_epochs}")
print(f"  Instantaneous batch size per device = {args.per_device_train_batch_size}")
print(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
print(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
print(f"  Total optimization steps = {args.max_train_steps}")

***** Running training *****
  Num examples = 2010
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 8
  Total optimization steps = 160


### Training

In [40]:
# args.add_canary = False
# Only show the progress bar once on each machine.
# progress_bar = tqdm(range(args.max_train_steps), disable=not accelerator.is_local_main_process)
completed_steps = 0
best_loss = 1000000
best_val_perplexity = float("inf")
save_path = f'models/{model.__class__.__name__}_gpt2_wikitext_pv.pt'
for epoch in range(args.num_train_epochs):
    ##################################################################
    # Train
    ##################################################################
    model.train()
    if accelerator.is_local_main_process:
        print(f"training epoch {epoch+1}")
    for step, batch in enumerate(tqdm(train_dataloader)):
        outputs = model(**batch)
        # loss = outputs.loss
        loss = outputs[0]
        loss = loss / args.gradient_accumulation_steps
        accelerator.backward(loss)
        if step % args.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            # progress_bar.update(1)
            completed_steps += 1
            
        if completed_steps >= args.max_train_steps:
            break  
            
    ##################################################################
    # Evaluation
    ##################################################################
    model.eval()
    losses = []

    if args.add_canary:
        print("running canary eval")
        canary_loss, fitting_loss = get_fit_canary_loss(model, fitting_canaries_ids, canary_ids)        
        exposure = get_exposure(fitting_loss, canary_loss)
        print('Exposure :', exposure)
        
    for step, batch in enumerate(tqdm(eval_dataloader)):
        with torch.no_grad():
            outputs = model(**batch)
            
        # loss = outputs.loss
        loss = outputs[0]
        losses.append(accelerator.gather(loss.repeat(args.per_device_eval_batch_size)))
        
    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]
    sorted_loss = sorted(losses)
    
    threshold = sorted_loss[int(0.1*len(losses))]
    if accelerator.is_local_main_process:
        print("threshold is: " , threshold.detach().item())
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")

    ################################################    
    #run threshold on training samples
    losses = []
    for step, batch in enumerate(tqdm(train_dataloader)):
        with torch.no_grad():
            outputs = model(**batch)
            
        # loss = outputs.loss
        loss = outputs[0]
        losses.append(accelerator.gather(loss.repeat(args.per_device_train_batch_size)))
          
    accelerator.wait_for_everyone()
    losses = torch.cat(losses)
    losses = losses[: len(train_dataset)]
    
    try:
        perplexity_train = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity_train = float("inf")

    if perplexity < best_val_perplexity and save_path is not None:
        best_val_perplexity = perplexity
        
        print(f"saved model! epoch {epoch}: perplexity: {best_val_perplexity}")
        torch.save(model.state_dict(), save_path)

    print(f"perplexity : {perplexity} perplexity train : {perplexity_train}")

training epoch 1


100%|██████████| 252/252 [02:53<00:00,  1.45it/s]


running canary eval
Exposure : 1.083239333053722


100%|██████████| 27/27 [00:06<00:00,  4.29it/s]


threshold is:  3.224740743637085


...

running canary eval
Exposure : 1.5929620442033978


100%|██████████| 27/27 [00:06<00:00,  4.26it/s]


threshold is:  3.1226134300231934


100%|██████████| 252/252 [01:00<00:00,  4.17it/s]


saved model! epoch 4: perplexity: 27.57423700825839
perplexity : 27.57423700825839 perplexity train : 32.31459358211376


In [41]:
# Set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

save_path = f'models/{model.__class__.__name__}_gpt2_wikitext_pv.pt'
state_dict = torch.load(save_path)
model.load_state_dict(state_dict)
model = model.to(device)

In [44]:
def generate(prompt, model, tokenizer, device, seed, max_length=100, temperature=0.7):
    input_ids = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt").to(device)
    output = model.transformer.generate(input_ids, max_length=max_length, temperature=temperature, num_return_sequences=1, do_sample=True)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

prompt = 'the secret number is'
# prompt = ' '
max_seq_len = 100
seed = 0
temperatures = [0.5, 0.7, 0.75, 0.8, 1.0]
for temperature in temperatures:
    generation = generate(prompt, model, tokenizer, device, seed, max_seq_len, temperature)
    print(f'{str(temperature)}\n{generation}\n')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0.5
the secret number is 100, so the value of the number " 0 " in the same sentence is 100. The same example is repeated in the same sentence with the same number. The value of the number " 0 " in the same sentence is 100. The same example is repeated in the same sentence with the same number. The same amount of space is shown in the list of " 0 ". The same example is repeated in the same sentence with the same number. The same amount of space is



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0.7
the secret number is " ', ", ", ", ", ", ", ", ", ", ", ", ", ", ", ", " ] ], ", ", ", ", ", ", ", ", ", ", ", ", ", ", ", ", ", ", ", ", ", ", " ), ", ", ", ", ", ",



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0.75
the secret number is.') ; } function test ( ) { var t = require ('test') ; t. prototype = '.'; t. setAttribute ('text') ; if (! t. hasOwnProperty ( '.') ||! t. hasOwnProperty ( '.') ) { var t = require ('test') ; t. prototype = t ; t. setAttribute ('text') ; } else { t =



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0.8
the secret number is not released. In many cases it is an unidentifiable number because it is not known whose initials are listed on that letter of the alphabet. The date and location of the secret number are not known which letters are used as numbers but rather are the dates and locations of " the first and last names of the members of the House of Commons " and " " the last names of the people of the Parliament ". The House of Commons has two numbers, one for each cabinet,

1.0
the secret number is a number that is not publicly known and the secret number for which there are no public records is "AID #2". After identifying the secret number and requesting the secret number of the person listed in The Telegraph's " Guardian of the day in 2007, it was discovered that at least one key official in Germany had been recorded in the UK in June 2007, for apparently attempting to kill off a UK resident. At the time of publication ; the date of the British public disclosure

